In [9]:
import pandas as pd
import influxdb_client
from influxdb_client import Point
from influxdb_client.client.write_api import SYNCHRONOUS
import glob

# Secrets
token = ""
url = "https://iwi-i-influx-db-01.hs-karlsruhe.de:8086"
bucket = "iot_gebaeude_a"
org = "Vorlesung"

# Client erstellen und Read/Write API anlegen
write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_api = write_client.write_api(write_options=SYNCHRONOUS)
query_api = write_client.query_api()

# Daten lesen
# NOTE: hier wird nur eine Datei von vielen eingelesen!
# data = pd.read_csv("./Daten/hka-a/hka-aqm-a014_2022_10_10.csv", sep=";")


file_paths = glob.glob("/Users/florian/Documents/github/study/IoT/hka-aqm-li/*.dat")

for file_path in file_paths:
    data = pd.read_csv(file_path, sep=";")

    # Daten erstellen
    # NOTE: Das muss natürlich nur einmal gemacht werden!
    for index, row in data.iterrows():
        for key, value in row.to_dict().items():
            point = Point(f"{index}_{key}")
            point.tag("sensor", f"{index}")
            point.field("value", value)
            write_api.write(bucket=bucket, org=org, record=point)

# Query erstellen
query = f"""from(bucket: "{bucket}")
 |> range(start: -10m)"""

# Query ausführen
tables = query_api.query(query, org=org)

# Ergebnis ausgeben
for table in tables:
    for record in table.records:
        print(record)




# Daten schreiben
# asdaskdaskd







NewConnectionError: <urllib3.connection.HTTPSConnection object at 0x140c45e50>: Failed to establish a new connection: [Errno 51] Network is unreachable